In [765]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [766]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.royallepage.ca/en/search/homes/?search_str=Ontario%2C+ON%2C+CAN&csrfmiddlewaretoken=7hyQJ3MRHrbS6nvhOv8JiR48aIOY8VgIyJszpJwGPm9PBHeSfwyLkioXigDdi2VH&property_type=7%2C8%2C5&house_type=102%2C103%2C104&features=&listing_type=&lat=50.926163435000035&lng=-84.74492999999995&upper_lat=&upper_lng=&lower_lat=&lower_lng=&bypass=&radius=&zoom=&display_type=gallery-view&travel_time=&travel_time_min=30&travel_time_mode=drive&travel_time_congestion=&da_id=&segment_id=&tier2=False&tier2_proximity=0&address=Ontario&method=homes&address_type=province&city_name=&prov_code=ON&school_id=&min_price=0&max_price=3000000&min_leaseprice=0&max_leaseprice=5000%2B&beds=0&baths=0&transactionType=SALE&archive_status=All&archive_timespan=6&sfproperty_type%5B7%5D=7&sfproperty_type%5B8%5D=8&sfproperty_type%5B5%5D=5&sfhouse_type%5B102%5D=102&sfhouse_type%5B103%5D=103&sfhouse_type%5B104%5D=104&keyword=&sortby="

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [767]:
# Go through all pages to collect data
house_listings = []
for x in range(1, 23):
    html = browser.html
    soup = bs(html, 'html.parser')
    links = soup.find_all(class_='address-1')
    
    # Get all address listings for buying    
    print('Gathering data from page:', x)
    for link in links:
        house_listings.append(link.a['href'])
    browser.find_by_css('.next').click()

# Number of listings
len(house_listings)


Gathering data from page: 1
Gathering data from page: 2
Gathering data from page: 3
Gathering data from page: 4
Gathering data from page: 5
Gathering data from page: 6
Gathering data from page: 7
Gathering data from page: 8
Gathering data from page: 9
Gathering data from page: 10
Gathering data from page: 11
Gathering data from page: 12
Gathering data from page: 13
Gathering data from page: 14
Gathering data from page: 15
Gathering data from page: 16
Gathering data from page: 17
Gathering data from page: 18
Gathering data from page: 19
Gathering data from page: 20
Gathering data from page: 21
Gathering data from page: 22


1012

In [768]:
browser.quit()

In [777]:
# Dependencies
import requests
import json

# Import the API key
from config import apiKey

# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)


# Visit each listing and grab relevant data
house_listing_info = []
for i in range(len(house_listings)):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 
    
    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    
    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text


    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    labels = building_feature_section.find_all('span', class_='label')
    values = building_feature_section.find_all('span', class_="value")
    

    wanted_features = ['Fireplace', 'Style', 'Cooling Type', 'Heating Type', 'Building Type']
    listed_features = []
    listed_values = []

    for i in range(len(labels)):
        listed_features.append(labels[i].text.strip(":"))
        listed_values.append(values[i].text.strip(":"))
        
    for i in range(len(wanted_features)):   
        if 'Fireplace' in listed_features:
            # Create index/value pairs for each feature
            fire_index = listed_features.index(wanted_features[0])
            fire_place = listed_values[fire_index]
            
        elif 'Style' in listed_features:
            style_index = listed_features.index(wanted_features[1])
            style_type = listed_values[style_index]
        
        elif 'Cooling Type' in listed_features:
            cooling_index = listed_features.index(wanted_features[2])
            cooling_type = listed_values[cooling_index]

        elif 'Heating Type' in listed_features:
            heating_index = listed_features.index(wanted_features[3])
            heating_type = listed_values[heating_index]
            
        elif 'Building Type' in listed_features:    
            building_index = listed_features.index(wanted_features[4])
            building_type = listed_values[building_index]
        else:
            fire_place = "No"
            style_place = ""
            cooling_type = ""
            heating_type = ""
            building_type = ""
            
        
    # Store in dictionary
    houses['address'] = address
    houses['price'] = price
    houses['number_of_beds'] = bed_count
    houses['number_of_baths'] = bath_count
    houses['style'] = style_type
    houses['cooling_type'] = cooling_type
    houses['heating_type'] = heating_type
    houses['fire_place'] = fire_place
    houses['building_type'] = building_type
    
    # Build target url with address and apikey
    string_search = address.split(",")[1:]
    new_string = (",").join(string_search).strip()                           
    
    target_url = f"https://api.geoapify.com/v1/geocode/search?text={new_string}&format=json&apiKey={apiKey}"
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()

    # Get coordinate information
    lat = geo_data["results"][0]["lat"]
    lon = geo_data["results"][0]["lon"]
    
    houses['lat'] = lat
    houses['lon'] = lon
        
    print(f"Collected data for: {address}")
    house_listing_info.append(houses)

browser.quit()

Collected data for: 6 HONEYBOURNE CRES, Markham, Ontario, L3P 1P3
Collected data for: 123 HEATH ST W, Toronto, Ontario, M4V 2Y6
Collected data for: 83 BOARDWALK DR, Toronto, Ontario, M4L 3X9
Collected data for: 2 LAKESIDE Drive, Grimsby, Ontario, L3M 2K7
Collected data for: #A -41 BROADVIEW AVE, Mississauga, Ontario, L5H 2S8
Collected data for: 2158 PINEVIEW DR, Oakville, Ontario, L6H 5M3
Collected data for: 31 BIRCH TREE TR, Brampton, Ontario, L6P 3W1
Collected data for: 20 PELLEGRINO RD, Brampton, Ontario, L7A 4M8
Collected data for: 36 JAMES WALKER AVE, Caledon, Ontario, L7C 1R5
Collected data for: 61 MIDSUMMER DR, Brampton, Ontario, L6P 3E4
Collected data for: #40 -74 COMPASS WAY, Mississauga, Ontario, L5G 4T8
Collected data for: 82 BLACKBURN AVENUE, Ottawa, Ontario, K1N 8A5
Collected data for: 1143 BANCROFT DR, Mississauga, Ontario, L5V 1B7
Collected data for: 815 GRACE ST, Newmarket, Ontario, L3Y 2L6
Collected data for: 98 ARDAGH ST, Toronto, Ontario, M6S 1Y7
Collected data for: 

AttributeError: 'NoneType' object has no attribute 'find'

In [778]:

print(f"Rows of data to work with: {len(house_listing_info)}")

Rows of data to work with: 248


In [ ]:
import csv

try:
    with open('./Resources/housing.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['lat', 'lon','address', 'price', 'number_of_beds', 'number_of_baths', 'style','building_type', 'cooling_type', 'heating_type', 'fire_place', ])
        writer.writeheader()
        for data in house_listing_info:
            writer.writerow(data)
except IOError:
    print("I/O error")